In [376]:
import csv
import pandas
import numpy as np
import operator
xtest = pandas.read_csv('bbb.test.csv', header = None)
xtrain = pandas.read_csv('bbb.train.csv',header = None)
ytest = pandas.read_csv('labels.test.csv',header = None)
ytrain = pandas.read_csv('labels.train.csv',header = None)
yytrain = ytrain.replace('nc', 1)
fytrain = yytrain.replace('c', 0)
yytest = ytest.replace('nc', 1)
fytest = yytest.replace('c', 0)

In [638]:
import plotly.graph_objects as go
def confusionM(truey, prey):
    v00, v01, v10, v11 = 0, 0, 0, 0
    for i in range(len(truey)):
        if truey[i] == 0:
            if prey[i] == 0:
                v00 += 1
            else:
                v10 += 1
        else:
            if prey[i] == 0:
                v01 += 1
            else:
                v11 += 1
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Confusion Matrix', 'True c(0)', 'True nc(1)', 'Total'],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[['Predict c(c)', 'Predict nc(1)', 'Total'], [v00, v10, v00+v10], [v01, v11, v01+v11], [v00+v01, v10+v11, ' ']],
                   fill_color='lavender',
                   align='left'))])
    fig.show()

In [695]:
def confusionM(truey, prey):
    v00, v01, v10, v11 = 0, 0, 0, 0
    for i in range(len(truey)):
        if truey[i] == 0:
            if prey[i] == 0:
                v00 += 1
            else:
                v10 += 1
        else:
            if prey[i] == 0:
                v01 += 1
            else:
                v11 += 1
    print('Confusion Matrix|', 'True c(0)', 'True nc(1)', 'Total')
    print('  Predicc c(0)  |',  v00,'       ', v01,'       ', v00+v01)
    print('  Predicc nc(1) |',  v10,'      ', v11,'      ', v10+v11)
    print('  Total         |',  v10+v00,'      ', v11+v01,'      ', v10+v11+v00+v01)

In [696]:
def PredictFunc(x, y, beta):
    res = {}
    res['error'] = 0
    res['prey'] = []
    res['truey'] = []
    for i in range(x.shape[0]):
        temp = x.iloc[i,:]
        tempty = y.iloc[i,0]
        temppy = PredOneRow(temp, beta)
        res['prey'].append(temppy)
        res['truey'].append(tempty)
        if tempty != temppy: 
            res['error'] += 1
    res['accu'] = 1 - res['error'] / x.shape[0]
    res['mse'] = res['error'] / x.shape[0]
    return res

def PredOneRow(x, beta):
    x = np.hstack((np.ones((1)), x.T))
    pre = pi_val(np.dot(x, beta))
    #print(x)
    if pre > 0.5:
        return 1
    return 0

In [628]:
def pi_val(input):
    output = 1 / (1 + np.exp(-input))
    return output

In [697]:
# Logistic Regression
def GradientD(x, y, stepsize, iters):
    beta = np.random.random_sample((x.shape[1],1))
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        ########################
        grad = np.dot(x.T, y - yy)
        beta = beta - stepsize*grad
    return beta

def LR(x, y, stepsize, iters):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD(x, y, stepsize, iters)
    return beta

def OutputFuncLo(beta, res):
    print('Coefficient: ', beta.flatten(), '\n')
    print('Error Num:', res['error'], '  Accuracy:', res['accu'])

#betaLo = LR(xtrain, fytrain, 0.01, 200)
#resLo = PredictFunc(xtest, fytest, betaLo)
#OutputFuncLo(betaLo, resLo)
confusionM(resLo['truey'], resLo['prey'])
t = 0
for i in resLo['truey']:
    if i == 0:
        t += 1
t


Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 0         0         0
  Predicc nc(1) | 27       44       71
  Total         | 27       44       71


27

In [659]:
def GradientD3(x, y, stepsize, iters, lam3):
    #beta = np.ones((x.shape[1],1))
    beta = np.random.random_sample((x.shape[1],1)) # 34*1
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = -2*np.dot(x.T, y - yy)+ lam3 * beta
        #print(grad)
        beta = beta - stepsize*grad
    return beta

def RR(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD3(x, y, stepsize, iters, lam3)
    return beta

def cvkfold(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = RR(xtrain_train, ytrain_train, 0.01, 100, lam3)
            restemp = test3(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse
 
def usebestlam(lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = 0
    for i in range(len(lambdalist)):
        if lambdalist[i] <= up:
            up = lambdalist[i]
            index = i
    bestlam = lambdapotential[index]
    beta1 = RR(xtrain, fytrain, 0.01, 100, bestlam)
    res = PredictFunc(xtest, fytest, beta1)
    print('Accuracy: ', res['accu'], ' Best Lambda: ', bestlam)
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

lambdapotential = [0.03, 0.01, 0.5]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold(data, lambdapotential, 5)
#print(lambdalist)
acc, lam = usebestlam(lambdapotential, xtrain, fytrain, xtest, fytest)


[ 1.       1.       1.      -0.16316  1.      -0.10169  0.99999 -0.15197
  1.      -0.19277  0.94055 -0.35151  0.95735 -0.29785  0.93719 -0.34412
  0.94486 -0.28106  0.90137 -0.43383  0.86043 -0.47308  0.82987 -0.5122
  0.8408  -0.47137  0.76224 -0.5837   0.65723 -0.68794  0.68714 -0.64537
  0.64727 -0.67226]
[ 1.       1.       0.35346 -0.13768  0.69387 -0.02423  0.68195 -0.03574
  0.55717 -0.06119  0.61836 -0.10467  0.62099 -0.06527  0.59361 -0.07289
  0.42271 -0.26409  0.58213  0.04992  0.49736 -0.08771  0.46241 -0.08989
  0.45008 -0.00564  0.39146 -0.09038  0.35588 -0.10306  0.32232 -0.08637
  0.28943 -0.083  ]
[ 1.       1.       0.7087  -0.24783  0.64348  0.04348  0.45217  0.38261
  0.65217  0.18261  0.5      0.26957  0.57826 -0.23043  0.50435  0.37826
  0.38696 -0.42609  0.36087 -0.26087  0.26957  0.11739  0.53246 -0.03845
  0.31304 -0.12174  0.4993  -0.04264  0.48348 -0.04448  0.64348 -0.25217
  0.50435  0.14783]
[ 1.       1.       0.32789  0.11042  0.1597   0.29308  0.1402   

In [667]:
from itertools import cycle
from copy import deepcopy


In [690]:
# 2/N (y-\beta x)^2
def f(x, y, beta): # 1*1
    n = x.shape[0]
    #print(y.shape, np.dot(x,beta).shape)
    tmp = y - np.dot(x,beta)
    return 2*np.dot(tmp.T, tmp) / n

def objective(x,y,beta,lam):
    n = x.shape
    tmp = y - np.dot(x,beta)
    obj = 2 * np.dot(tmp,tmp) / n + lam * np.sum(np.abs(beta))
    return obj


def LASSO_proximal_gradient(x, y, lam, stepsize = 1, max_iter=100, thres = 0.01):
    #beta = 0.5
    p, n = x.shape
    beta = np.random.random_sample((n,1)) 
    #print(beta.shape)
    beta_last = beta
    xty = np.dot(x.T, y) / n # 1/N x^Ty
    # print(y.shape) x:33*280, y:280*1, xty = 33*1
    xtx = np.dot(x.T, x) / n # 1/N x^Tx 33*33
    lamStep = lam / stepsize
    objV = np.empty(max_iter, dtype=float)
    for k in range(max_iter):
        while True:
            grad = np.dot(xtx, beta)- xty # 33*1
            #print(grad.shape)
            z = beta - grad/stepsize # 33*1
            beta_tmp = np.sign(z) * np.maximum(np.abs(z)-lamStep, 0)# 33*1
            #print(beta_tmp.shape)
            beta_diff = beta_tmp - beta
            if f(x, y, beta_tmp) <= f(x, y, beta) + np.dot(grad.T, beta_diff) + stepsize/2 * np.sum(beta_diff**2):
                break
            stepsize = stepsize / 0.5
        beta_old = deepcopy(beta)
        beta = deepcopy(beta_tmp)
        
        objV[k] = objective(x,y,beta,lam)
        if k > 0 and abs(objV[k] - obj[k-1]) < thres:
            break
    return beta#, obj[:k]


def lasso(x, y, lamrange):
    p, n = x.shape
    betas = np.empty((n, len(lamrange)), dtype=float)
    for i, lam in enumerate(lamrange):
        beta = LASSO_proximal_gradient(x, y, lam)
        betas[:,i] =  beta
    return lam, betas


lasso(xtrain, fytrain, [0.1])

ValueError: shapes (280,1) and (280,1) not aligned: 1 (dim 1) != 280 (dim 0)